# Data Cleaning

### Imports

In [1]:
import pandas as pd
import numpy as np

### Reading the file that I created using web scraping

In [2]:
df=pd.read_csv('logic_immo_scrap_complete.csv')

In [3]:
df.shape

(4240, 6)

In [4]:
df.head()

,Superficie,Rooms,Bedrooms,Price,Location,description
0,33m²,1 p.,-1,328 600 €,Paris 19 (75019)|Secrétan - Jaurès...,Ref de l'annonce\r\n:Grand Studio\r\nParking\r...
1,122m²,5 p.,4 p.,1 940 000 €,Paris 04 (75004)\r\nParis 4ème,Ref de l'annonce\r\n:158\r\nParking\r\nTerrass...
2,68m²,4 p.,3 p.,830 000 €,Paris 10 (75010)\r\nParis 10ème,Ref de l'annonce\r\n:2679339\r\nTerrasse/Balco...
3,36m²,2 p.,1 p.,524 000 €,Paris 18 (75018)\r\nAbbesses,Ref de l'annonce\r\n:0555\r\nEtage du bien\r\n...
4,32m²,1 p.,-1,375 000 €,Paris 15 (75015)\r\nParis 15ème,Ref de l'annonce\r\n:162373805\r\nChauffage\r\...


### In this Data Cleaning notebook, I will try to do the following things:
    - "Location" extract only the district
    - "Superficie" parsing
    - Bedrooms check for -1 values
    - Parsing of description column: I will extract the following features: Parking, Cellar, floor building,floor apartment, patio/balcony, Heating, nb_bathrooms, Renovated
    - Price parsing

-> Let's try to extract the district from the column 'Location'.

In [5]:
#Location extract only the district
import re
df["Loc_Paris"]=df.Location.apply(lambda x: re.findall(r'\d{5}',x)[0] )
df.Loc_Paris.value_counts()

75018    481
75015    424
75016    391
75017    322
75020    285
75013    233
75014    209
75011    209
75019    196
75006    178
75007    174
75008    174
75010    166
75012    150
75005    110
75004    107
75009    100
75003     94
75116     89
75002     88
75001     60
Name: Loc_Paris, dtype: int64

In [6]:
#It seems that some districts are misspelled.
df[df.Loc_Paris=='75116']

,Superficie,Rooms,Bedrooms,Price,Location,description,Loc_Paris
62,17m²,2 p.,1 p.,365 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:d2117f\r\nChauffage\r\n:I...,75116
117,29m²,1 p.,-1,457 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:W-02I86Q\r\nEtage du bien...,75116
123,177m²,5 p.,3 p.,1 930 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:W-02GTRV\r\nNombre d'étag...,75116
124,201m²,6 p.,3 p.,3 150 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:W-02HFSG\r\nEtage du bien...,75116
125,143m²,4 p.,2 p.,2 665 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:W-02EE9H\r\nChauffage\r\n...,75116
...,...,...,...,...,...,...,...
4005,420m²,10 p.,5 p.,12 900 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:Henri Martin\r\nRénové\r\...,75116
4047,362m²,8 p.,5 p.,5 950 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:1018PSN\r\nChauffage\r\n:...,75116
4055,201m²,6 p.,3 p.,2 500 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:577VHFDH\r\nChauffage\r\n...,75116
4057,89m²,3 p.,2 p.,1 030 000 €,Paris 16 (75116)\r\nParis,Ref de l'annonce\r\n:546\r\nChauffage\r\n:Cent...,75116


In [7]:
# it should be '75016' instead of '75116'
df.Loc_Paris.replace('75116','75016',inplace=True)
df.Loc_Paris.value_counts()

75018    481
75016    480
75015    424
75017    322
75020    285
75013    233
75011    209
75014    209
75019    196
75006    178
75008    174
75007    174
75010    166
75012    150
75005    110
75004    107
75009    100
75003     94
75002     88
75001     60
Name: Loc_Paris, dtype: int64

In [8]:
#transform the feature loc_Paris 
df.Loc_Paris=df.Loc_Paris.apply(lambda x: int(x)-75000)
#I Want the column Loc_Paris to be treated as a categorical variable
df['Loc_Paris']=df.Loc_Paris.apply(lambda x: str(x))

-> Superficie Parsing

In [9]:
#Superficie parsing
df['m2']=df.Superficie.apply(lambda x: int(x.split('m²')[0]))
df.m2

0        33
1       122
2        68
3        36
4        32
       ... 
4235    128
4236     56
4237    155
4238    107
4239     43
Name: m2, Length: 4240, dtype: int64

-> Rooms Parsing

In [10]:
df.Rooms.value_counts()

2 p.     1169
3 p.      924
1 p.      726
4 p.      590
5 p.      467
6 p.      198
8 p.       82
7 p.       67
9 p.        7
12 p.       3
10 p.       3
14 p.       1
30 p.       1
16 p.       1
11 p.       1
Name: Rooms, dtype: int64

In [11]:
df['nb_rooms']=df.Rooms.apply(lambda x: x.replace('p.','').strip())
df.nb_rooms.value_counts()
#outliers spotted we will deal with them in the EDA section

2     1169
3      924
1      726
4      590
5      467
6      198
8       82
7       67
9        7
12       3
10       3
30       1
14       1
11       1
16       1
Name: nb_rooms, dtype: int64

-> Bedrooms Parsing

In [12]:
df.Bedrooms=df.Bedrooms.apply(lambda x: x.replace('p.','').strip())
df['nb_bedrooms']=df.Bedrooms.apply(lambda x: x.replace('-1', '0'))                                                     

In [13]:
df.nb_bedrooms.value_counts()

1     1369
2     1031
0      735
3      676
4      356
5       60
6        9
12       1
8        1
7        1
10       1
Name: nb_bedrooms, dtype: int64

-> Parsing of description: extract features: Parking, Cellar, floor building, floor apartment, patio/balcony, Heating, nb_bathrooms, Renovated ?

In [14]:
#We create a funciton to extract features from the column description

def extract_features(x,feature):
    for i in range(len(df)):
        if feature in str(x):
            return (x.split(f'{feature}\r\n:')[1].split('\r')[0])
        else:
            return 0

In [15]:
#Extracting the number of floor of the building
df["building_floor"]=df.description.apply(lambda x: extract_features(x,'bâtiment'))

#Extracting where the apartment is located in the building
df["apartment_floor"]=df.description.apply(lambda x: extract_features(x,'bien'))

#Extracting the number of bathrooms
df["nb_bath"]=df.description.apply(lambda x: extract_features(x,'bain'))

#Extracting the presence or absence of a Cellar
df["Cellar"]=df.description.apply(lambda x: 1 if "cave" in str(x).lower() else 0)

#Extracting the presence or absence of a balcony or a patio
df["balc_patio"]=df.description.apply(lambda x: extract_features(x,'Balcon'))

#Extracting the type of Heating
df["Heating"]=df.description.apply(lambda x: extract_features(x,'Chauffage'))

#Finding if the apartment was renovated
df["Renovated"]=df.description.apply(lambda x: 1 if "rénové" in str(x).lower() else 0)

#Extracting if the presence or absence of a Parking
df["Parking"]=df.description.apply(lambda x: 1 if "parking" in str(x).lower() else 0)

-> Price parsing

In [16]:
#price parsing
df=df[~(df.Price=='Nous consulter')]
df['SalePrice']=df.Price.apply(lambda x: int(x.replace('€',"").replace(' ','')))

In [17]:
df.head()

,Superficie,Rooms,Bedrooms,Price,Location,description,Loc_Paris,m2,nb_rooms,nb_bedrooms,building_floor,apartment_floor,nb_bath,Cellar,balc_patio,Heating,Renovated,Parking,SalePrice
0,33m²,1 p.,-1,328 600 €,Paris 19 (75019)|Secrétan - Jaurès...,Ref de l'annonce\r\n:Grand Studio\r\nParking\r...,19,33,1,0,14,1er,1,0,0,0,0,1,328600
1,122m²,5 p.,4,1 940 000 €,Paris 04 (75004)\r\nParis 4ème,Ref de l'annonce\r\n:158\r\nParking\r\nTerrass...,4,122,5,4,4,2e,1,1,Terrasse,0,0,1,1940000
2,68m²,4 p.,3,830 000 €,Paris 10 (75010)\r\nParis 10ème,Ref de l'annonce\r\n:2679339\r\nTerrasse/Balco...,10,68,4,3,3,0,1,0,Terrasse,0,0,0,830000
3,36m²,2 p.,1,524 000 €,Paris 18 (75018)\r\nAbbesses,Ref de l'annonce\r\n:0555\r\nEtage du bien\r\n...,18,36,2,1,0,2e,1,1,0,0,0,0,524000
4,32m²,1 p.,-1,375 000 €,Paris 15 (75015)\r\nParis 15ème,Ref de l'annonce\r\n:162373805\r\nChauffage\r\...,15,32,1,0,9,8e,0,1,0,Au gaz,0,0,375000


### Now that we have done some Feature Engineering and Data cleaning, I think it could be interesting to explore the values of the different features of the dataframe.

In [18]:
#Loc_Paris
df.Loc_Paris.value_counts()

18    481
16    479
15    424
17    322
20    285
13    233
14    209
11    209
19    195
6     178
7     174
8     172
10    166
12    150
5     110
4     107
9     100
3      94
2      88
1      60
Name: Loc_Paris, dtype: int64

Everything seems normal.

In [19]:
#building_floor
df.building_floor.value_counts().sort_values()

33         1
19         1
21         1
20         1
26         1
28         2
27         2
1900       2
22         3
18         3
29         4
17         5
16         6
32         7
30         9
13         9
31        10
15        10
12        29
11        38
1         43
14        50
2         59
10        77
9        101
3        170
8        203
4        239
7        355
5        539
0       1116
6       1140
Name: building_floor, dtype: int64

The 0 values correspond to missing values. 
Indeed it seems that 1117 properties advertisements did not specify the number of floors of the building. To deal with missing values we can do 3 things:
1.Delete the rows with missing values
2.Delete the column
3.Impute the mean number of floors for each district. 
I tried the three solutions and it seems that the first one works best.
But I have another idea that could work well. I think separate the number of buildings into four categories: Small, Average, High, Tower. Let's do that and we will check later in the EDA if it was a good idea.

-> First solution

In [20]:
#df.building_floor=df.groupby('Loc_Paris').building_floor.transform(lambda x: x.replace(0,int(x.median()))).apply(lambda x: int(x))
#df.building_floor.value_counts()

-> Third Solution

In [21]:
#Drop the columns where we do not know the number of floors of the building.
#df=df[~(df.building_floor==0)]
#Drop the rows with 1900 floors, because this is not possible
df=df[~(df.building_floor=='1900')]
#df.building_floor.value_counts()

-> My idea

In [22]:
#Function giving four categories of buildings in Paris
def height(x):
    if x ==0:
        return 'Average'
    elif x<4:
        return 'Small'
    elif x<=9:
        return 'Average'
    elif x<=12:
        return 'High'
    else:
        return 'Tower'
        

In [23]:
df['building_height']=df.building_floor.apply(lambda x: height(int(x)))
df.building_height.value_counts()

Average    3693
Small       272
High        144
Tower       125
Name: building_height, dtype: int64

In [24]:
#apartment floor
df.apartment_floor.value_counts()

0         891
1er       699
2e        617
4e        501
3e        494
5e        415
6e        290
7e        132
8e         67
9e         40
11e        17
10e        17
15e         9
18e         6
13e         5
12e         5
17e         3
16e         3
28e         2
24e         2
29e         2
25e         2
21e         2
30e         2
32767e      1
20e         1
32e         1
1930e       1
19e         1
23e         1
1900e       1
27e         1
14e         1
22e         1
26e         1
Name: apartment_floor, dtype: int64

Some rows with wrong values such as '32767' or '1900e' or '1930e'. We should drop them.
Same problem that with the building floor, it seems that 546 property advertisements did not specify at which floor the apartment is located in the building. After looking online, I found out that these properties were at the ground floor of the building. Therefore let us create another fearure.

In [25]:
#Imputation
#df.apartment_floor=df.groupby('Loc_Paris').apartment_floor.transform(lambda x: x.replace(0,x.value_counts().idxmax()))
#Replace the remaining 0 by the most frequent occurences in the all districts
#df.apartment_floor=df.apartment_floor.transform(lambda x: x.replace(0,df.apartment_floor.value_counts().idxmax()))
#df.apartment_floor.value_counts()

In [26]:
#Drop rows with wrong values such as '32767' or '1900e' or '1930e'
df=df[~(df.apartment_floor=='1900e')]
df=df[~(df.apartment_floor=='32767e')]
df=df[~(df.apartment_floor=='1930e')]
#Creation of the "ground floor" feature
df['Ground_floor']=df.apartment_floor.apply(lambda x: 1 if x==0 else 0)
df.Ground_floor.value_counts()

0    3340
1     891
Name: Ground_floor, dtype: int64

In [27]:
#I will drop the rows where we do not know both the height of the building and the floor of the apartment
df= df[~((df.apartment_floor==0) & (df.building_floor==0))]

In [28]:
#transform apartment floor and building floor into numerical features
df['apartment_floor']=df.apartment_floor.apply(lambda x: int(str(x).replace('e','').replace('r','')))
df['building_floor']=df.building_floor.apply(lambda x: int(x))

In [29]:
#Cellar
df.Cellar.value_counts()

1    2045
0    1840
Name: Cellar, dtype: int64

In [30]:
#balcony and patio
df.balc_patio.value_counts()

0                   2837
Balcon               686
Terrasse             214
Terrasse, balcon     148
Name: balc_patio, dtype: int64

In [31]:
#replace 0 values with Unknown since we do  not know what type of Heating is used
df.Heating=df.Heating.apply(lambda x: str(x).replace('0','Unknown'))

In [32]:
#Create two new features for Heating: "individual" and "Central", since both are the most important classes.
df['Heating_Central']=df.Heating.apply(lambda x: 1 if 'Central' in str(x) else 0)
df['Heating_Individual']=df.Heating.apply(lambda x: 1 if 'Individuel' in str(x) else 0)
df.Heating_Central.value_counts()


0    3111
1     774
Name: Heating_Central, dtype: int64

In [33]:
#Parking
df.Parking.value_counts()

0    3464
1     421
Name: Parking, dtype: int64

In [34]:
#Drop the rows with unknown for m2 feature
df=df[~(df.m2==-1)]

In [35]:
df.columns

Index(['Superficie', 'Rooms', 'Bedrooms', 'Price', 'Location', 'description',
       'Loc_Paris', 'm2', 'nb_rooms', 'nb_bedrooms', 'building_floor',
       'apartment_floor', 'nb_bath', 'Cellar', 'balc_patio', 'Heating',
       'Renovated', 'Parking', 'SalePrice', 'building_height', 'Ground_floor',
       'Heating_Central', 'Heating_Individual'],
      dtype='object')

### Now I will create another feature that could be interesting "Last floor". I think this feature can have an impact on the price. I will verify this assumption in the Exploratory Data Analysis.

In [36]:
#Let us create one other feature that could be interesting: 'Last_etage'
df['Last_floor']= df.apply(lambda x: 1 if int(x.building_floor)==int(x.apartment_floor) else 0,axis=1)



I will finish this Data cleaning part by looking at the types of the features that I created.

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3868 entries, 0 to 4239
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Superficie          3868 non-null   object
 1   Rooms               3868 non-null   object
 2   Bedrooms            3868 non-null   object
 3   Price               3868 non-null   object
 4   Location            3868 non-null   object
 5   description         3868 non-null   object
 6   Loc_Paris           3868 non-null   object
 7   m2                  3868 non-null   int64 
 8   nb_rooms            3868 non-null   object
 9   nb_bedrooms         3868 non-null   object
 10  building_floor      3868 non-null   int64 
 11  apartment_floor     3868 non-null   int64 
 12  nb_bath             3868 non-null   object
 13  Cellar              3868 non-null   int64 
 14  balc_patio          3868 non-null   object
 15  Heating             3868 non-null   object
 16  Renovated           3868

In [38]:
#I want the feature Loc_Paris to be categorical
df['district']=df.Loc_Paris.apply(lambda x: str(x)+'e')



In [39]:
df.to_csv('House_price_cleaned.csv',index=False)